In [19]:
import json

with open("yelp1.json", 'r') as data_file:
    data = json.load(data_file)

new_file = []
for cuisine, rest_info in data.items():
    tmp = rest_info
    for rest in tmp:
        rest["cuisine"] = cuisine
    new_file.append(tmp)
    
with open("data.json", 'w') as data_file:
    data = json.dump(new_file, data_file)

In [31]:
with open("data.json", "r") as data_file:
    data = json.load(data_file)

new_file = []
for element in data:
    for rest in element:
        new_file.append(rest)

with open("yelp1.json", "w") as data_file:
    json.dump(new_file, data_file)

In [1]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import datetime

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

### create table ###

table = dynamodb.create_table(
    TableName='yelp-restaurants-chatbot',

    KeySchema = [
        {
            'AttributeName': 'cuisine',
            'KeyType': 'HASH' # The values of two partition keys could be the same
        },
        {
            'AttributeName': 'id',
            'KeyType': 'RANGE'
        }
    ],

    AttributeDefinitions=[
        {
            'AttributeName': 'cuisine',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'id',
            'AttributeType': 'S'
        }
    ],

    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)


Table status: CREATING


In [14]:
### import JSON to items ###
from datetime import *
table = dynamodb.Table('yelp-restaurants-chatbot')

with open("yelp_1000.json", 'r') as json_file:
    data = json.load(json_file, parse_float = decimal.Decimal)
    for restaurant in data:
        cuisine = "cuisine" if "cuisine" not in restaurant else restaurant["cuisine"]
        ID = "id" if "id" not in restaurant else restaurant["id"]
        name = "restaurant" if "name" not in restaurant else restaurant["name"]
        price = "$" if "price" not in restaurant else restaurant["price"]
        rating = 3 if "rating" not in restaurant else restaurant["rating"] 
        review_count = 0 if "review_count" not in restaurant else restaurant["review_count"]
        location = "location is not provided by Yelp." if "location" not in restaurant else restaurant["location"]
        time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        Item = {
                'cuisine': cuisine,
                'id': ID,
                'name': name,
                'price': price,
                'rating': rating,
                'reviewCount': review_count,
                'location': location,
                'insertedAtTimestamp': time
            }       
        print(Item)

#         table.put_item(
#             Item = {
#                 'cuisine': cuisine,
#                 'id': ID,
#                 'name': name,
#                 'price': price,
#                 'rating': rating,
#                 'reviewCount': review_count,
#                 'location': location,
#                 'insertedAtTimestamp': time
#             }       
#         )

{'cuisine': u'chinese', 'reviewCount': 98, 'name': u'Xifu Food', 'rating': Decimal('4.5'), 'price': u'$', 'id': u'rPTX2J5KGDkN1QhPD-QOvg', 'insertedAtTimestamp': '2019-04-24 17:58:35', 'location': {u'city': u'Brooklyn', u'display_address': [u'318 Livingston St', u'Brooklyn, NY 11217'], u'country': u'US', u'address2': u'', u'address3': None, u'state': u'NY', u'address1': u'318 Livingston St', u'zip_code': u'11217'}}
{'cuisine': u'chinese', 'reviewCount': 191, 'name': u'Han Dynasty', 'rating': Decimal('4.0'), 'price': u'$$', 'id': u'Q1TpfbuPwKPFrNFyu-u2_g', 'insertedAtTimestamp': '2019-04-24 17:58:35', 'location': {u'city': u'Brooklyn', u'display_address': [u'1 Dekalb Ave', u'Brooklyn, NY 11201'], u'country': u'US', u'address2': u'', u'address3': None, u'state': u'NY', u'address1': u'1 Dekalb Ave', u'zip_code': u'11201'}}
{'cuisine': u'chinese', 'reviewCount': 322, 'name': u"Auntie Guan's Kitchen 108", 'rating': Decimal('3.5'), 'price': u'$$', 'id': u'dDXtga5Kgo5B1twdezcZJA', 'insertedAt

In [26]:
### Elastic Search ### 
import json

with open("yelp1.json", "r") as data_file:
    data = json.load(data_file)


i = 1
new_file = []
for element in data:
    tmp = element
    if "price" in tmp:
        tmp.pop("price")
    if "coordinates" in tmp:
        tmp.pop("coordinates")
    if "location" in tmp:
        tmp.pop("location")
    if "categories" in tmp:
        tmp.pop("categories")
    dic_inst = {"index": {"_index": "restaurants", "_type": "Restaurant", "_id": str(i)}}
    new_file.append(dic_inst)
    new_file.append(tmp)
    i += 1

with open("yelp_es.json", "w") as data_file:
    json.dump(new_file, data_file)


In [1]:
### Elastic Search - ML Prediction ### 
import json

with open("yelp_ml_preES.json", "r") as data_file:
    data = json.load(data_file)


i = 1
new_file = []
for element in data:
    tmp = element
    if "price" in tmp:
        tmp.pop("price")
    if "rating" in tmp:
        tmp.pop("rating")
    if "review_count" in tmp:
        tmp.pop("review_count")
    if "location" in tmp:
        tmp.pop("location")

    dic_inst = {"index": {"_index": "predictions", "_type": "Prediction", "_id": str(i)}}
    new_file.append(dic_inst)
    new_file.append(tmp)
    i += 1

with open("yelp_ml_es.json", "w") as data_file:
    json.dump(new_file, data_file)
